# # “Synthetic Data for Label Balancing: Should You Use It and How?”

# Introduction

The developments in the field of Generative AI architectures also means higher quality tabular synthetic data (and it keeps getting better). Having worked with synthetic data, I’ve come across many articles that promote it as a catch-all solution for various obstacles (e.g. machine learning applications, privacy, ....). While synthetic data definitely is useful and has bright prospects (?), its usefulness and relevance is not always clearly assessed. An example of this is, and also the inspiration for writing this article, is the article provided by Synthetic Data Vault (SDV) titled: "Can you use synthetic data for label balancing?" (https://sdv.dev/blog/synthetic-label-balancing/).

SDV’s article addresses a common challenge in classification: imbalanced target labels. Synthetic data is proposed as a 'compelling solution' for this problem compared to more traditional approaches without any empirical evidence.  Although you definitely can use synthetic data for label balancing (to answer the question of the article), the key question is whether you **should** use synthetic data and how it compares to state-of-the-art techniques.

This article addresses the question of the article by SDV on whether you should use synthetic data for imbalanced classification tasks. It also demonstrates how to set up a proper cross-validation pipeline to prevent data leakage between training and holdout sets when applying resampling techniques.


TODO:
- Run the file and get results
- Introduction
- re-write section just below

## Imports

Notably, to ensure a proper cross validation procedure, I use a pipeline. Specifically, the pipeline from imb_learn is used over sklearn's

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt     

import lightgbm as lgb

from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.pipeline import Pipeline
from imblearn.combine import SMOTEENN

from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import StratifiedKFold, cross_validate, train_test_split

from sdv.metadata import Metadata
from sdv.single_table import CTGANSynthesizer, TVAESynthesizer

from helper_functions import NoiseSampler, SDVSampler, SDVENN, display_scores

## Settings

Finally, we define our random seed for reproducability, define the metrics we will use to evaluate our models, and define the CV folds.

In [2]:
RANDOM_STATE = 2

SCORINGS = ['f1', 'roc_auc', 'precision', 'recall']

CV_FOLDS = StratifiedKFold(n_splits = 5, random_state = RANDOM_STATE, shuffle = True)

## Data exploration

For this analysis, the creditcard dataset will be used from Kaggle (https://www.kaggle.com/datasets/mlg-ulb/creditcardfraud), containing transactions and whether they were fraudulent or not. To preserve privacy, most of the features are principal components derived from the original dataset. The goal is to predict whether a transaction is fraudulent or not, making it a classification task. Here the 'Class' variable indicates whether the transactions is fraudulent or not.


In [3]:
creditcard = pd.read_csv('./data/creditcard.csv')

Next, we see how our target labels are distributed.

In [4]:
# Target label distribution rounded to 2 decimal places
round(creditcard['Class'].value_counts(normalize = True) * 100, 2)

Class
0    99.83
1     0.17
Name: proportion, dtype: float64

Naturally, the amount of non-fraudulent transactions outweigh the number of fraudulent transactions resulting in an imbalanced classification task. In this situation, the dataset is highly imbalanced, with  only 0.17% of transactions being fraudulent. This imbalance can pose challenges for machine learning models, which may become biased toward predicting the majority class. To address this issue, resampling methods are popular. These are methods to balance the dataset by increasing the minority class or by decreasing the majority class. And sometimes a combination of both, also called hybrid sampling.





**re-write for clarity below**

One of the techniques used to increase the minority class, i.e. oversampling, is through synthetic data as SDV suggests. But how does this approach perform and more importantly compare to other techniques?

As you might be thinking (and I was too), this idea probably isn't very novel, and indeed similar research exists in the literature. The work of Adiputra and Wanchai (2024) caught my eye, which compares similar data-level approaches. However after snooping around in the repository containing the code for their research, I noticed in the validation synthetic data is generated **before** performing cross-validation, which is a common pitfall leading to data leakage and biased results.

**Still explain/display what incorrect CV is below. Or at a later section. I can just mention here: 'why this is an issue and how this works will be displayed later/below**

## Data splitting

Next step is to split the data. We split the data into a CV set and a test set. We stratify on the target variable to ensure an even split across sets. The CV set will be used to perform cross validation on and the test will be the untouched data to showcase the effect of improper cross validation procedure and how this generalizes to truely unseen data. 

In [5]:
X = creditcard.drop('Class', axis = 1)
y = creditcard['Class']

X_cv, X_test, y_cv, y_test = train_test_split(X, y, test_size = 0.2, stratify = y,
                                                    random_state = RANDOM_STATE)

## Incorrect cross validation procedure 

Firstly, displaying what an incorrect cross validation setup looks like. Suppose we wish to use Random OverSampler, which balances the data by duplicating minority class instances, to randomly oversample the fraudulent transactions. A common mistake that is made, also made by Adiputra and Wanchai (2024), which inspired this blog, is to perform this resampling before splitting the data. 

This cross validation mistake is easily made and looks like this in code:

In [ ]:
# We resample the data before performing cross-validation
ros = RandomOverSampler(random_state = RANDOM_STATE)
X_res, y_res = ros.fit_resample(X_cv, y_cv)

# Feed the already resampled data into cross validation
cv_score = cross_validate(
    lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1),
    X_res, y_res, cv = CV_FOLDS, scoring = SCORINGS, n_jobs = -1
)

# Calculate the mean f1 score
cv_score = cv_score['test_f1'].mean()
print(f"Cross validation score: {cv_score:.4f}")

Cross validation score: 0.9999


This approach leads to near perfect scores and this seems great, but how do these results translate to truly unseen data: the test set we have not used in the RandomOverSampler.


In [7]:
# We define the model and fit it to the resampled cv data
lgbm_classifier = lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, 
                                     verbose = -1)
lgbm_classifier.fit(X_res, y_res)

# predictions on the unseen data
preds = lgbm_classifier.predict(X_test)
test_score = f1_score(y_test, preds)
print(f"Test score: {test_score:.4f}")

Test score: 0.8442


The test results are noticeably lower than the cross-validation results, indicating that the cross-validation score obtained this way is not representative of the performance on truly unseen data. Furthermore, you might also want to tune hyperparameters during cross-validation. In this case, hyperparameters are selected based on an incorrect cross-validation procedure and overfit in that process, damaging generalization of your models.



## What is going wrong?

Although the difference may seem small, resampling before cross-validation can significantly affect the generalizability of results. The main idea behind creating a train and holdout split is to evaluate the model on data that it has not seen before. Cross-validation applies this same exact concept across multiple folds of the data to limit the variance of the results that come from making one single random split. In each fold, the holdout set is used as a separate unseen set of data to test the model on. Therefore, for each fold in the cross-validation procedure, no information from the holdout set should leak into the train set to ensure that it is still unseen.

By using ROS before cross validation in this example, we create duplicates of fraud instances across the ENTIRE dataset. During cross-validation, these duplicated fraud instances can end up in the holdout set of a fold whereas the original ends up in the train set of a fold (or vice versa). As a result, the holdout set does not consist of truly unseen data and the model is tasked to predict the outcome of an observation it has already seen during training. 

To demonstrate this, we look at the exact duplicates across folds when ROS is used before and during cross-validation.

**Add visual here????**

In [9]:
# Isolate fraudulent transactions
X_pos = X[y == 1]

# Determine the proportion of duplicates in the positive class
prop_dupl = X_pos.duplicated(keep = False).sum() / X_pos.shape[0]
print(f"Duplicates amongst fraudulent transactions in the entire dataset: {prop_dupl:.4f}")

Duplicates amongst fraudulent transactions in the entire dataset: 0.0650


This indicates that approx. 6.5% of fraud instances are not unique. Using stratified folds, we would expect to see a similar percentage of duplicates for each fold throughout cross-validation. 

First, the number of duplicated fraud instances across folds are displayed for the data that is resampled before cross-validation, i.e. the X_res and y_res from earlier. Specifically, we look into how many duplicates there are between train and holdout for each fold.

In [11]:
# Again, resampling the data before cross-validation
ros = RandomOverSampler(random_state = RANDOM_STATE)
X_res, y_res = ros.fit_resample(X_cv, y_cv)

# to track the proportion of duplicates
dupl_percentages = []

# Loop over each fold using the already resampled data
for i, (train_index, hd_index) in enumerate(CV_FOLDS.split(X_res, y_res)):
    
    # Obtain train and holdout sets for the current fold
    X_train, X_hd = X_res.iloc[train_index], X_res.iloc[hd_index]
    y_train, y_hd = y_res.iloc[train_index], y_res.iloc[hd_index]
    
    # Isolate the fraud instances
    train_pos = X_train[y_train == 1]
    hd_pos = X_hd[y_hd == 1]
    
    # Check the observations in holdout that are duplicates of the training set for fold i
    mask = hd_pos.apply(tuple, axis = 1).isin(train_pos.apply(tuple, axis = 1))
    duplicates = hd_pos[mask]
    
    # Determine the proportion of holdout observation that are duplicates
    prop_dupl = duplicates.shape[0] / hd_pos.shape[0] * 100
    dupl_percentages.append(prop_dupl)
    
print(f"Proportion of duplicates in holdout set across folds: {np.mean(dupl_percentages):.2f}%")


Proportion of duplicates in holdout set across folds: 100.00%


When resampling is performed before cross-validation, the average percentage of duplicates across folds reaches 100%. This means that every fraud instance in the holdout set is a duplicate of an observation in the training set, explaining the near perfect F1-scores observed earlier.

While the high duplication rate is largely attributed to the severe class imbalance in this dataset, the issue may manifest more subtly in other contexts. Moreover, data leakage will not always appear as exact duplicates for all synthesizers. For example, a synthesizer might avoid creating exact duplicates but still generate numerous highly similar observations. Therefore, one should also understand the behavior of the synthesizer used.

## Well... how do you resample the correct way?

There are multiple ways to do it. Namely, iterating over each fold in a loop and applying the transformations to the train and holdout set separately. The manual approach below is used to get a feel of how the transformations should look like and what is exactly is going wrong in the previous setup. Afterwards, a pipeline will be used for a more efficient setup. 

The correct way to resample is performed by iterating over the unresampled data and only then applying resampling WITHIN each fold as opposed to beforehand.

In [ ]:
# track the test scores for each fold and average them afterwards
cv_results = []

# Loop over the folds of the data that is NOT resampled beforehand
for i, (train_index, hd_index) in enumerate(CV_FOLDS.split(X_cv, y_cv)):
    
    # Obtain train and holdout sets for fold i
    X_train, X_hd = X_cv.iloc[train_index], X_cv.iloc[hd_index]
    y_train, y_hd = y_cv.iloc[train_index], y_cv.iloc[hd_index]
    
    # Applying the ROS to the training set WITHIN each fold as opposed to beforehand 
    ros = RandomOverSampler(random_state = RANDOM_STATE)
    X_train_res, y_train_res = ros.fit_resample(X_train, y_train)
    
    # Fit the model to the resampled train set and predict the holdout set that is not resampled
    lgb_clf = lgb.LGBMClassifier(random_state = RANDOM_STATE, n_jobs = -1, verbose = -1)
    lgb_clf.fit(X_train_res, y_train_res)
    cv_preds = lgb_clf.predict(X_hd)
    
    # Calculate and append F1 score for the current fold
    cv_score = f1_score(y_hd, cv_preds)
    cv_results.append(cv_score)
    
print(f"F1 score with correct CV procedure: {np.mean(cv_results):.4f}")

F1 score with correct CV procedure: 0.8384


Although the score is less impressive than the first method, it more accurately reflects the results on the truly unseen holdout set of 0.8442. And how is this reflected in the duplicates in the holdout set?

In [ ]:
# Again, we track the proportion of duplicates 
dupl_percentages = []

for i, (train_index, hd_index) in enumerate(CV_FOLDS.split(X_cv, y_cv)):
   
    # Obtain train and holdout sets for fold i
    X_train, X_hd = X_cv.iloc[train_index], X_cv.iloc[hd_index]
    y_train, y_hd = y_cv.iloc[train_index], y_cv.iloc[hd_index]
    
    # WITHIN each fold, we resample the training set
    ros = RandomOverSampler(random_state = RANDOM_STATE)
    X_res, y_res = ros.fit_resample(X_train, y_train)
    
    # Only then do we isolate the positive class
    train_pos = X_res[y_res == 1]
    hd_pos = X_hd[y_hd == 1]
    
    # Check to see the observations in holdout that are duplicates of the training set
    mask = hd_pos.apply(tuple, axis = 1).isin(train_pos.apply(tuple, axis = 1))
    duplicates = hd_pos[mask]
    
    # determine the proportion of holdout observation that are duplicates
    prop_dupl = duplicates.shape[0] / hd_pos.shape[0] * 100
    dupl_percentages.append(prop_dupl)
    
print(f"Proportion of duplicates in test set across folds: {np.mean(dupl_percentages):.2f}%")

Proportion of duplicates in test set across folds: 6.35%


**Stayed here with check**

With this, we can see that the proportion of duplicates across folds resembles that of the one we obtained from the entire dataset, indicating that resampling within each fold leads to folds that are more representative. Furthermore, the effect of improper cross validation procedure in this example with ROS is clear in this situation due to the large amount of exact duplicates. However, depending on the severity of class imbalance, the synthetic data generator used, and the degree to which the generator may overfit, this data leakage issue might be less obvious. 

Moreover, this issue of data leakge between train and holdout is not something that only pertains to resampling methods in imbalanced classification tasks. This also applies to other, perhaps more subtle, forms of data leakage, such as feature engineering methods that use information from observations that will end up in the holdout set during cross validation. Think of scaling for instance. Ideally, a scaler should be fit only on the training set and then applied to both training and holdout sets, repeating this for each fold in cross-validation. 

Though not all feature engineering has to lead to data leakage between train and holdout. For instance, the encoding of a categorical gender variable from male/female to binary, which typically is fine since the information is contained within the rows. However, this might get a bit tricky when working with a high cardinality categorical variable where certain values are quite rare meaning that they might appear in the hold out set but not in the train set. For this reason, it's important to be aware of what transformations are being applied, when they are applied, and what information they rely on. Best practice is to perform all transformations within cross-validation rather than beforehand to prevent data leakage. 

## Now that we know how to evaluate resampling techniques during cross-validation, what is the best way to deal with the imbalance?

As previously mentioned, imbalanced datasets can lead to biased results towards the larger class. Ways to deal with this can be categorized into data level and algorithm level approaches. Data level approaches are techniques that artificially over/under/hybrid sample observations to balance the dataset. Whereas algorithm level approaches don't do anything to the underlying data, but to the algorithm to deal with this imbalance in the dataset

The data level approaches we use firstly include ROS, noise injection, and synthetic data as covered in SDV's article. While ROS is a common oversampling technique, noise injection is less familiar in practice, and since the article doesn't detail the noise generation process, I assume values are randomly sampled from a normal distribution. Synthetic data generation creates artificially generated observations of the minority class by learning the underlying data patterns. In this scenario, the synthetic data should ideally resemble real observations to help improve model performance. This resemblance is called having high fidelity. However, excessively high fidelity risks creating exact copies of the original data, which can result from the generator overfitting to the data.

The techniques we use to generate synthetic data are Synthetic Minority Over-sampling TEchnique (SMOTE), Conditional Tabular Generative Adversarial Networks (CTGAN), and Tabular Variational Auto Encoder (TVAE), for which we will use SDV's library for the latter two. SMOTE generates synthetic examples of the minority class by interpolating between observations based on their feature space distance, whereas CTGAN and TVAE use neural networks to learn data representations and generate samples without relying on explicit distance-based interpolation.

Furthermore, we also use an algorithm-level approach: cost-sensitive learning (CSL). CSL applies different weights to observations so that the model learns to minimize the costs associated with misclassifying observations based on their importance. In this context, fraud instances are given higher weights, meaning a larger penalty is assigned to misclassifying fraudulent transactions. CSL is particularly effective when you know the exact costs of misclassification. Since the true misclassification costs are unknown in this context, we use inverse class frequency as the weighting scheme, which assigns higher weights to the rarer fraud class.

Given that the folds are stratified, we assume the that the assigned weights calculated on the entire train set is equal to the weights for each fold. This will result in the following weights being assigned:

In [21]:
w1, w2 = X.shape[0] / (2 * np.bincount(y)) 
print(f"Majority class weight: {w1:.4f}, minority class weight: {w2:.4f}")

Majority class weight: 0.5009, minority class weight: 289.4380


Finally, inspired by Adiputra and Wanchai (2024), we also use hybrid sampling. Specifically, we will throw SMOTE-ENN and their proposed CTGAN-ENN into the mix, which combine oversampling with downsampling using Edited Nearest Neighbors, which removes observations from the majority class that are close to the decision boundary. CTGAN-ENN has been shown to provide improved performance over CSL and other over- and hybrid sampling techniques.


# Data level approaches/Pipeline display (i will skip over the individual codes)

Previously, we have performed cross validation by manually looping over the folds. Although this provides a lot of control, there is an easier and cleaner way to do it. We can use a Pipeline to perform the same operations. A Pipeline offers a more concise and is an easy to use alternative. The pipelines used will ensure that transformations are always fit on the train set within each fold and subsequently transforming the holdout set, reducing the risk of data leakage.

In this article, we use the Pipeline from the imblearn package. While scikit-learn’s Pipeline can also handle transformations between training and test sets, imblearn’s version supports samplers, making it more suitable for our workflow.


## Random OverSampling pipeline example 1


First, we define the classifier to be used in the pipeline (and in other pipelines). In this example, we arbitrarily choose the LightGBM classifier, but you could replace this with any classifier of your choosing. Next, we construct the pipeline, which includes the preprocessing steps (e.g., our RandomOverSampler, which can be directly passed into the pipeline) followed by the classifier. Finally, we pass this pipeline into the cross_validate function, which handles resampling within each fold, which was previously performed manually, and evaluates the model’s performance across folds using the metrics defined in our SCORINGS list.


In [ ]:
# Define the LightGBM classifier to be used in the pipeline
lgb_clf = lgb.LGBMClassifier(random_state=RANDOM_STATE, n_jobs=-1, verbose=-1)

# Define the sampling pipeline
pipeline_ros = Pipeline(
    [('ros', RandomOverSampler(random_state = RANDOM_STATE)),
     ('lgbm', lgb_clf)] 
)

cv_score_ros = cross_validate(pipeline_ros, X, y, cv = CV_FOLDS, scoring = SCORINGS, 
                               n_jobs = -1)

## Noise injection

**not shown in blog, only result**

In [ ]:
pipeline_noise = Pipeline(
    [('noise_sampler', NoiseSampler(random_state = RANDOM_STATE)),
     ('lgbm', lgb_clf)]
)

cv_score_noise = cross_validate(pipeline_noise, X, y, cv = CV_FOLDS, scoring = SCORINGS, 
                                n_jobs = -1)

## SMOTE

**not shown in blog, only result**

In [ ]:
pipeline_smote = Pipeline(
    [('scaler', StandardScaler()),
     ('smote', SMOTE(random_state = RANDOM_STATE)),
     ('lgbm', lgb_clf)]
)

cv_score_smote = cross_validate(pipeline_smote, X, y, cv = CV_FOLDS, scoring = SCORINGS, 
                                n_jobs = -1)

## SDV Generators

In order to use SDV's synthetic data generators, we first need to define the metadata of the dataset used. This can simply be done with with the following code.

In [6]:
# We first define the metadata for the SDV synthesizers
metadata = Metadata.detect_from_dataframe(X)

It is also possible to update the metadata manually if you wish. Furthermore, SDV offers a wide variety of synthetic data generators, as well as many functions and parameters to try to improve the quality of the synthetic data. Namely, SDV provides useful preprocessing tools and constraints (i.e., deterministic rules the synthetic data should adhere to) to incorporate domain knowledge and business rules. Since the other methods used in this article work out of the box, I will not be applying any of these methods and will compare them in the same way.

### CTGAN Pipeline example 2



Integrating CTGAN into a pipeline is a bit more involved since there is no standard support for it. Therefore, we have to build our custom resampling class, which we can pass into the pipeline.

We build a BaseSampler class we can use for all resampling procedures that are not natively supported in the Pipeline from imb_learn (i.e. CTGAN, TVAE, and the custom noise sampling procedure). This BaseSampler contains all the base functiontality that is consistent across these samplers and ensures that during cross-validation, resampling happens within each fold while keeping the training and holdout sets properly separated.

In [14]:
class BaseSampler(BaseEstimator):
    """Base class for all custom samplers.
    """
    
    def __init__(self, random_state):
        self.random_state = random_state
    
    def fit_resample(self, X, y):
        """Fit the sampler to the data and return the resampled data"""
        return self.resample(X, y)
    
    def _get_num_samples(self, y):
        """Get the number of samples to generate for the minority class for this binary
        classification problem."""
        y_values = y.value_counts()
        num_samples = y_values[0] - y_values[1]
        
        return num_samples  

    def resample(self, X, y):
        """Resample the data using the specified sampling method"""
        
        X_train = X.copy()
        y_train = y.copy()
        
        num_samples = self._get_num_samples(y_train)

        # Obtain the minority class that needs to be upsampled
        X_minority = X_train[y_train == 1]
        
        X_upsampled = self._generate_syn_data(X_minority, num_samples)

        # Updating indepent variables and the target variable
        X_sampled = pd.concat([X_train, X_upsampled], axis = 0)
        y_sampled = pd.concat([y_train, pd.Series(np.ones(num_samples))], axis = 0)

        return X_sampled, y_sampled
    

NameError: name 'BaseEstimator' is not defined

Next, we define the SDVSampler class for the CTGAN and TVAE synthesizers, based on BaseSampler as its parent class. This class takes the synthesizer generator in the __init__ method and has a custom _generate_syn_data() function to resample using the SDV synthesizers.

In [ ]:
class SDVSampler(BaseSampler):  
    
    def __init__(self, generator, metadata, random_state):
        super().__init__(random_state)
        self.generator = generator
        self.metadata = metadata
       
    def _generate_syn_data(self, X_minority, num_samples):
        """Generate synthetic data using the SDV synthesizer."""
        
        X_resample = X_minority.copy()
        
        # Creating a new instance of the synthesizer within each fold
        synthesizer = self.generator(self.metadata)

        synthesizer.fit(X_resample)
        X_sds = synthesizer.sample(num_samples)
        
        return X_sds

The final pipeline for CTGAN then looks as follows:

In [ ]:
from sdv.single_table import CTGANSynthesizer

# Define the CTGAN pipeline using our custom SDVSampler class
pipeline_ctgan = Pipeline(
    [('ctgan', SDVSampler(CTGANSynthesizer, metadata, random_state = RANDOM_STATE)),
     ('lgbm', lgb_clf)]
)

cv_score_ctgan = cross_validate(pipeline_ctgan, X, y, cv = CV_FOLDS, scoring = SCORINGS, 
                               n_jobs = -1)

### TVAE Synthesizer


In [ ]:
pipeline_tvae = Pipeline(
    [('tvae', SDVSampler(TVAESynthesizer, metadata, random_state = RANDOM_STATE)),
     ('lgbm', lgb_clf)]
)

cv_score_tvae = cross_validate(pipeline_tvae, X, y, cv=CV_FOLDS, scoring = SCORINGS,
                               n_jobs = -1)

# Hybrid Sampling

## SMOTE ENN


In [36]:
pipeline_smoteenn = Pipeline(
    [('scaler', StandardScaler()),
     ('smote enn', SMOTEENN(random_state = RANDOM_STATE)),
     ('lgbm', lgb_clf)]
)

cv_score_smoteenn = cross_validate(pipeline_smoteenn, X, y, cv = CV_FOLDS, scoring = SCORINGS, 
                                n_jobs = -1)


KeyboardInterrupt: 

## CTGAN ENN


In [ ]:
pipeline_ctganenn = Pipeline(
    [('ctgan', SDVENN(CTGANSynthesizer, metadata, random_state = RANDOM_STATE)),
     ('lgnm', lgb_clf)]
)

cv_score_ctganenn = cross_validate(pipeline_ctganenn, X, y, cv = CV_FOLDS, 
                                   scoring = SCORINGS, n_jobs = -1)

KeyboardInterrupt: 

# Algorithm level approach

## Cost sensitive learning



In [23]:
# Define the sampling pipeline
pipeline_csl = Pipeline(
    [('LGB', lgb.LGBMClassifier(class_weight= 'balanced', random_state = RANDOM_STATE,
                                n_jobs = -1, verbose = -1))]
)

cv_score_csl = cross_validate(pipeline_csl, X, y, cv=CV_FOLDS, scoring = SCORINGS,
                               n_jobs = -1)

# Results

In [ ]:
cv_scores = [cv_score_noise, cv_score_ros, cv_score_smote, cv_score_ctgan, cv_score_tvae, 
             cv_score_smoteenn, cv_score_ctganenn, cv_score_csl]   

In [25]:
# Complete the scores of 
r = display_scores(cv_scores, SCORINGS)
r

,F1,Roc_auc,Precision,Recall
Baseline,0.000,0.499,0.000,0.000
Noise,0.708,0.974,0.626,0.819
ROS,0.837,0.975,0.862,0.815
SMOTE,0.654,0.961,0.538,0.837
CTGAN,0.764,0.971,0.721,0.817
TVAE,0.763,0.967,0.715,0.821
Gaussian Copula,0.847,0.978,0.906,0.797
CSL,0.839,0.975,0.843,0.835


Gaussian copula provides the best F1, which is kind of surprising. This is likely very dataset dependent and to rule out/for synthetic data, this process should in actuality be repeated for multiple datasets, which I don't have the time to do. Perhaps I should add a quick section: 'Understanding the data', since the dataset at hand might be over simplistic.

However, the Gaussian Copula provides the worsy recall of all methods (and best precision). For the other methods, this is more aligned with each other. So, it would also still be dependent on how you want to use the model and the cost of misclassification.

TODO:
- Highlight how CTGAN-ENN compares against SMOTE-ENN and all in general
- Based on the mediocre results from SMOTE experiment with fewer examples